In [ ]:
from bql import Service, Request
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Connect to Bloomberg BQL
bq = Service()

# ----------------------------
# Step 1: FETCH INTRADAY DATA
# ----------------------------

tickers = {
    "TY1": "TY1 Comdty",         # 10Y Treasury Future
    "SOFR": "SR3 Comdty",        # SOFR Future
    "SWAP5Y": "USSW5 Curncy"     # 5Y USD Swap Rate
}

start_dt = "2024-06-01T09:30:00"
end_dt = "2024-06-01T16:00:00"
interval = "5MIN"

# BQL queries for each ticker
qry_ty = bq.execute(Request("BQL", f"get(px_last) for('{tickers['TY1']}') with(dates=range('{start_dt}','{end_dt}'), interval={interval})"))
qry_sofr = bq.execute(Request("BQL", f"get(px_last) for('{tickers['SOFR']}') with(dates=range('{start_dt}','{end_dt}'), interval={interval})"))
qry_swap = bq.execute(Request("BQL", f"get(last_price) for('{tickers['SWAP5Y']}') with(dates=range('{start_dt}','{end_dt}'), interval={interval})"))

# Convert to DataFrames
df_ty = qry_ty[0].df().rename(columns={"value": "TY1_price"}).set_index("time")
df_sofr = qry_sofr[0].df().rename(columns={"value": "SOFR_fut"}).set_index("time")
df_swap = qry_swap[0].df().rename(columns={"value": "SWAP5Y_rate"}).set_index("time")

# Merge and clean
df = df_ty.join(df_swap, how='outer').join(df_sofr, how='outer')
df = df.fillna(method='ffill').dropna()

# ----------------------------
# Step 2: SIMULATE EXPOSURE & DV01
# ----------------------------

np.random.seed(42)
df["notional_exposure"] = np.random.choice([10e6, 20e6, 30e6], size=len(df))
df["TY1_dv01"] = df["notional_exposure"] * (-0.0000085)
df["SWAP5Y_dv01"] = df["notional_exposure"] * 0.00005
df["SOFR_dv01"] = df["notional_exposure"] * 0.00002
df["net_dv01"] = df["TY1_dv01"] + df["SWAP5Y_dv01"] + df["SOFR_dv01"]
DV01_THRESHOLD = 500
df["hedge_needed"] = (abs(df["net_dv01"]) > DV01_THRESHOLD).astype(int)

# ----------------------------
# Step 3: TRAIN CLASSIFICATION MODEL
# ----------------------------

features = ["TY1_price", "SWAP5Y_rate", "SOFR_fut", "notional_exposure"]
X = df[features]
y = df["hedge_needed"]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
df.loc[X_test.index, "hedge_pred"] = model.predict(X_test)

# ----------------------------
# Step 4: STRATEGY RECOMMENDATION
# ----------------------------

df["hedge_strategy"] = "No hedge needed"
df.loc[(df["hedge_pred"] == 1) & (df["net_dv01"] > 0), "hedge_strategy"] = "Sell TY1 or Pay 5Y Swap"
df.loc[(df["hedge_pred"] == 1) & (df["net_dv01"] < 0), "hedge_strategy"] = "Buy TY1 or Receive 5Y Swap"

# ----------------------------
# Step 5: OUTPUT PREVIEW
# ----------------------------

report = classification_report(y_test, df.loc[X_test.index, "hedge_pred"], output_dict=True)
output_preview = df[["TY1_price", "SWAP5Y_rate", "SOFR_fut", "net_dv01", "hedge_pred", "hedge_strategy"]].tail(10)

# Show results
print("Model Performance:")
print(report)
print("\nSample Hedge Recommendations:")
print(output_preview)